# To do list
- downloading code - call function (provided by Jiawei)  
- downloading code - OSM (Dongqi to figure out)  
- pre-processing   
- merge to create static driver  

In [2]:
import requests
import getpass, pprint, time, os, cgi, json
import geopandas as gpd
import osmnx as ox
import rasterio
from geocube.api.core import make_geocube
from datetime import datetime, timezone
import pandas as pd
from pyproj import Transformer, CRS
from shapely.geometry import box, Point
from pathlib import Path
import os
import pickle
import numpy as np
import pandas as pd
from pyproj import Proj, transform
import osmnx as ox
import rasterio
from geocube.api.core import make_geocube
from util.get_osm import *
from util.get_sst import download_sst
from util.get_geo_nasa import *
from util.loc_dom import domain_location, domain_nest, write_cfg
from util.check_empty import check_empty
from util.create_static import *
from util.pre_process_tif import *
from util.pre_process_tif import main as process_main
import configparser
import ast
import sys
from urllib.request import urlretrieve
import warnings
## supress warnings
## switch to other actions if needed
warnings.filterwarnings("ignore")

In [2]:
# https://appeears.earthdatacloud.nasa.gov/products

## config

In [3]:
#--------------------------------------------------------------------------------#
#--------------------------------------------------------------------------------#
# read namelist
settings_cfg = configparser.ConfigParser(inline_comment_prefixes='#')
config = configparser.RawConfigParser()
prefix = "prefix"#sys.argv[1]
namelist =  f"./JOBS/{prefix}/INPUT/namelist.static-{prefix}"
config.read(namelist)
## [case]
case_name =  ast.literal_eval(config.get("case", "case_name"))[0]
origin_time = ast.literal_eval(config.get("case", "origin_time"))[0]
# local projection (unit: m)
config_proj = ast.literal_eval(config.get("case", "config_proj"))[0]
# use WGS84 (EPSG:4326) for centlat/centlon
default_proj = ast.literal_eval(config.get("case", "default_proj"))[0] 

## [domain configuration]
ndomain = ast.literal_eval(config.get("domain", "ndomain"))[0]
centlat = ast.literal_eval(config.get("domain", "centlat"))
centlon = ast.literal_eval(config.get("domain", "centlon"))
dx = ast.literal_eval(config.get("domain", "dx"))
dy = ast.literal_eval(config.get("domain", "dy"))
dz = ast.literal_eval(config.get("domain", "dz"))
nx = ast.literal_eval(config.get("domain", "nx"))
ny = ast.literal_eval(config.get("domain", "ny"))
nz = ast.literal_eval(config.get("domain", "nz"))
z_origin = ast.literal_eval(config.get("domain", "z_origin"))
ll_x = ast.literal_eval(config.get("domain", "ll_x"))
ll_y = ast.literal_eval(config.get("domain", "ll_y"))

## [required tif files]
sst = ast.literal_eval(config.get("geotif", "sst"))
dem = ast.literal_eval(config.get("geotif", "dem"))
lu = ast.literal_eval(config.get("geotif", "lu"))

dem_start_date = ast.literal_eval(config.get("geotif", "dem_start_date"))[0]
dem_end_date = ast.literal_eval(config.get("geotif", "dem_end_date"))[0]

lu_start_date = ast.literal_eval(config.get("geotif", "lu_start_date"))[0]
lu_end_date = ast.literal_eval(config.get("geotif", "lu_end_date"))[0]


## [tif files for urban canopy]
bldh = ast.literal_eval(config.get("urban", "bldh"))
bldid = ast.literal_eval(config.get("urban", "bldid"))
pavement = ast.literal_eval(config.get("urban", "pavement"))
street = ast.literal_eval(config.get("urban", "street"))

## [tif files for plant canopy]
sfch = ast.literal_eval(config.get("plant", "sfch"))



# specify the directory of tif files
# users can provide their own tif files
# otherwise will download from NASA or OSM
static_tif_path = f'./JOBS/{case_name}/INPUT/'
output_path = static_tif_path.replace("INPUT","OUTPUT")
tmp_path = static_tif_path.replace("INPUT","TMP")
## create folders for temporary tif files and final netcdf outputs
if not os.path.exists(tmp_path):
    print("Create tmp folder")
    os.makedirs(tmp_path)
if not os.path.exists(output_path):
    print("Create output folder")
    os.makedirs(output_path)
## these dictionanries only pass keys 
tif_geotif_dict = dict(config.items('geotif'))
tif_urban_dict = dict(config.items('urban'))
tif_plant_dict = dict(config.items('plant'))



In [4]:

for i in range(0,ndomain):
    if i == 0:
        case_name_d01 = case_name+"_N01"
        dom_cfg_d01 = {'origin_time': origin_time,
                    'centlat': centlat[i],  
                    'centlon': centlon[i],
                    'dx': dx[i],
                    'dy': dy[i],
                    'dz': dz[i],
                    'nx': nx[i],
                    'ny': ny[i],
                    'nz': nz[i],
                    'z_origin': z_origin[i],
                    }
        
        tif_dict_d01 = {}
        for keys in tif_geotif_dict.keys():
            tif_dict_d01[keys] = ast.literal_eval(config.get("geotif", keys))[i]
        for keys in tif_urban_dict.keys():
            tif_dict_d01[keys] = ast.literal_eval(config.get("urban", keys))[i]
        for keys in tif_plant_dict.keys():
            tif_dict_d01[keys] = ast.literal_eval(config.get("plant", keys))[i]
        # configure domain location information
        dom_cfg_d01 = domain_location(default_proj, config_proj,  dom_cfg_d01)
        # generate static driver 
#         dom_cfg_d01 = generate_palm_static(case_name_d01, config_proj, tif_proj, dom_cfg_d01, tif_dict_d01)
        
        #--------------------------------------------------------------------------------#
        ## first check if we need to download data from online sources
        ## SST data
        if tif_dict_d01["sst"]=="online":
            download_sst(case_name, origin_time, static_tif_path)
        ## DEM and land use (currently only for NASA online data sets)
        ## prepare dictionaries
        geodata_name_dict = {}
        output_format_dict  = {}
        start_date_dict = {}
        end_date_dict = {}
        if tif_dict_d01["dem"]=="online":
            geodata_name_dict["DEM"] = ["SRTMGL1_NC.003",]
            output_format_dict["DEM"] = "geotiff"
            # NASA DEM data only available for these dates
            start_date_dict["DEM"] = dem_start_date
            end_date_dict["DEM"] = dem_end_date
        if tif_dict_d01["lu"]=="online":
            # https://lpdaac.usgs.gov/documents/101/MCD12_User_Guide_V6.pdf
            geodata_name_dict["Land_Use"] = ["MCD12Q1.006",]
            output_format_dict["Land_Use"] = "geotiff"
            # User to choose the start/end date
            start_date_dict["Land_Use"] = lu_start_date
            end_date_dict["Land_Use"] = lu_end_date
        ## download data for NASA AρρEEARS API only
        if tif_dict_d01["dem"]=="online" or tif_dict_d01["lu"]=="online":
            area_radius = np.max([dx[i]*nx[i], dy[i]*ny[0]])/2 # units=metre
            default_buffer_ratio = 1.2 # used to multiply area_radius avoid areas becoming smaller than required after reproject
            api = 'https://appeears.earthdatacloud.nasa.gov/api/'  # Set the AρρEEARS API to a variable
            task_type = 'area'   # this is the only type used in this script
            
            download_nasa_main(api, geodata_name_dict, centlon[i], centlat[i], area_radius, default_proj, \
                       default_buffer_ratio, start_date_dict,end_date_dict, output_format_dict,case_name,static_tif_path)
        if tif_dict_d01["bldh"]=="online" or tif_dict_d01["bldid"]=="online":
            get_osm_building(centlat[i], centlon[i], area_radius, static_tif_path, case_name, i)
        if tif_dict_d01["pavement"]=="online":
            get_osm_street(centlat[i], centlon[i], area_radius, static_tif_path, case_name, i)
    ## for child domains 
    else:    
        #--------------------------------------------------------------------------------#
        # downloading data for nested domains
        #--------------------------------------------------------------------------------#
        dom_cfg_nest = {'origin_time': origin_time,
                    'dx': dx[i],
                    'dy': dy[i],
                    'dz': dz[i],
                    'nx': nx[i],
                    'ny': ny[i],
                    'nz': nz[i],
                    'z_origin': z_origin[i],
                    }
        ll_x_nest, ll_y_nest = ll_x[i], ll_y[i]

        dom_cfg_nest = domain_nest(config_proj, dom_cfg_d01['west'], dom_cfg_d01['south'], ll_x_nest, ll_y_nest,dom_cfg_nest)

        tif_dict_nest = {}
        for keys in tif_urban_dict.keys():
            tif_dict_nest[keys] = ast.literal_eval(config.get("urban", keys))[i]
        for keys in tif_plant_dict.keys():
            tif_dict_nest[keys] = ast.literal_eval(config.get("plant", keys))[i]
        area_radius_nest = np.max([dx[i]*nx[i], dy[i]*ny[i]])/2 # units=metre

        if tif_dict_nest["bldh"]=="online" or tif_dict_nest["bldid"]=="online":
                get_osm_building(dom_cfg_nest["centlat"], dom_cfg_nest["centlon"], area_radius_nest, static_tif_path, case_name, i)
        if tif_dict_nest["pavement"]=="online":
                get_osm_street(dom_cfg_nest["centlat"], dom_cfg_nest["centlon"], area_radius_nest, static_tif_path, case_name, i)
        # save domain configuration for later - creating static drivers
        with open(f'{tmp_path}{case_name}_cfg_N0{i+1}.pickle', 'wb') as dicts:
            pickle.dump(dom_cfg_nest, dicts, protocol=pickle.HIGHEST_PROTOCOL)

Retrieving SST data from OPeNDAP
SST file exist


Enter NASA Earthdata Login Username: 
(Register here if you haven't, https://www.earthdata.nasa.gov/eosdis/science-system-description/eosdis-components/earthdata-login)
Username: dli84
Enter NASA Earthdata Login Password:  ········


User credental successfully verified!
AρρEEARS currently supports 157 products.
Checking DEM dataset (SRTMGL1_NC.003)
SRTMGL1_NC.003 in the datasets
Checking Land_Use dataset (MCD12Q1.006)
MCD12Q1.006 in the datasets
Dataset check successful.
Checking layer information for the DEM dataset (SRTMGL1_NC.003)
Checking layer information for the Land_Use dataset (MCD12Q1.006)


There are multipe layers in the dataset of Land_Use (MCD12Q1.006).Please choose one of the layers below:
['LC_Prop1', 'LC_Prop1_Assessment', 'LC_Prop2', 'LC_Prop2_Assessment', 'LC_Prop3', 'LC_Prop3_Assessment', 'LC_Type1', 'LC_Type2', 'LC_Type3', 'LC_Type4', 'LC_Type5', 'LW', 'QC']
 'LC_Type1'


Projection check done. Start submitting jobs.


Files already exists in ./JOBS/prefix/INPUT/prefix_DEM_SRTMGL1_NC.003. Enter 1 to continue task downloading or 0 to skip this task. 0
Files already exists in ./JOBS/prefix/INPUT/prefix_Land_Use_MCD12Q1.006. Enter 1 to continue task downloading or 0 to skip this task. 0


Retrieving OSM building data...


SystemExit: No OSM data retrievied. Please check your configuration.

In [21]:
from pyproj import Transformer

In [28]:
dom_cfg_d01 = domain_location(default_proj, config_proj,  dom_cfg_d01)


In [32]:
domain_nest(config_proj, dom_cfg_d01['west'], dom_cfg_d01['south'], ll_x_nest, ll_y_nest,dom_cfg_nest)

{'origin_time': '2021-01-01 00:00:00 +00',
 'dx': 81.0,
 'dy': 81.0,
 'dz': 18.0,
 'nx': 216,
 'ny': 216,
 'nz': 36,
 'z_origin': 0.0,
 'centlat': -43.5144493944088,
 'centlon': 172.5734219756331,
 'west': 1556814.2944948424,
 'east': 1574229.2944948424,
 'south': 5173258.021971742,
 'north': 5190673.021971742,
 'lon_w': 172.46500036350716,
 'lon_e': 172.68156466637436,
 'lat_s': -43.59239751932636,
 'lat_n': -43.43639649804719}

In [5]:
#--------------------------------------------------------------------------------#
#--------------------------------------------------------------------------------#
# preprocess tif files 
process_main(prefix)

DEM tif file for Domain N01 exists
LU tif file for Domain N01 exists
Building tif file for Domain N01 exists
pavement tif file for Domain N01 exists
street tif file for Domain N01 exists
DEM tif file for Domain N02 exists
LU tif file for Domain N02 exists
Building tif file for Domain N02 exists
pavement tif file for Domain N02 exists
street tif file for Domain N02 exists


In [8]:
#--------------------------------------------------------------------------------#
#--------------------------------------------------------------------------------#
# create static driver 
for i in range(0,ndomain):
    if i==0:
        dom_cfg_d01 = generate_palm_static(case_name,tmp_path, i, config_proj, dom_cfg_d01)
    else:
        with open(f'{tmp_path}{case_name}_cfg_N0{i+1}.pickle', 'rb') as dicts:
             dom_cfg_nest = pickle.load(dicts)
        dom_cfg_nest = generate_palm_static(case_name,tmp_path, i, config_proj, dom_cfg_nest)

@@@@dem (221, 221)
south -24254.315127795307 west -24095.573671532988
lon_min 1548771.2290874715 lon_max 1602231.2290874715
lat_min 5154123.771045684 lat_max 5207583.771045684
@@@@zt (200, 200)
Number of grid points x, y = 200, 200
Reading land use data
south -24254.315127795307 west -24095.573671532988
lon_min 1548771.2290874715 lon_max 1602231.2290874715
lat_min 5154123.771045684 lat_max 5207583.771045684
Reading building ID
south -24254.315127795307 west -24095.573671532988
lon_min 1548771.2290874715 lon_max 1602231.2290874715
lat_min 5154123.771045684 lat_max 5207583.771045684
Reading building height
south -24254.315127795307 west -24095.573671532988
lon_min 1548771.2290874715 lon_max 1602231.2290874715
lat_min 5154123.771045684 lat_max 5207583.771045684
Reading surface height for vegetation height
Reading pavement
south -24254.315127795307 west -24095.573671532988
lon_min 1548771.2290874715 lon_max 1602231.2290874715
lat_min 5154123.771045684 lat_max 5207583.771045684
Reading stre

# debugging